In [35]:
import pandas as pd
from etl import ETL

In [36]:
etl = ETL()
data_json = etl.extract()
print(data_json)

None


In [37]:
data = pd.DataFrame(data_json)

In [38]:
print(data.head())

Empty DataFrame
Columns: []
Index: []


In [39]:
data.head()

""
